In [1]:
import nibabel as nib
import numpy as np
import os

flat_data = []
all_data = []
ages = []
sexes = []
dementiaClass = []
sessions = []
non_zeros = np.load('non_zeros.npy')
fails = 0
rootDir = 'data'
dirName, subdirList, fileList = os.walk(rootDir).next()
for f in sorted(fileList):
    if "MR1" not in f: continue
    if "txt" in f: 
        sessionID = f.split('.')[0]
        with open(rootDir + "/" + f) as metadataFile:
            sessions.append(sessionID)
            for line in metadataFile:
                if "AGE:" in line:
                    ages.append(int(list(filter(None, line.split()))[1]))
                elif "M/F:" in line:
                    sexes.append(list(filter(None, line.split()))[1])
                elif "CDR" in line:
                    try:
                        dementiaClass.append(int(list(filter(None, line.split()))[1]))
                    except:
                        fails += 1
                        dementiaClass.append(-1)
                    break
                
    elif "img" in f:
        data = nib.load(rootDir + "/" + f).get_data()
        # print data.shape
        all_data.append(data)
        #flat_data.append(data.flatten(order='C')[non_zeros])
        #flat_data.append(data.flatten(order='F')[non_zeros])

#flat_data_np = np.array(flat_data)
all_data = np.squeeze(np.array(all_data))
ages = np.array(ages, dtype='float64')
dementiaClass = np.array(dementiaClass)
                                
# WARNING: UNCOMMENTING THE FOLLOWING LINES WILL OVERWRITE YOUR BACKUPS
#np.save('sessions', np.array(sessions))
#np.save('flat_data2', flat_data_np)
#np.save('ages2', ages)
#np.save('sexes2', sexes)
#np.save('dementia', dementiaClass)

### Creating histograms from data

In [2]:
# finding items, that are equal everywhere
def filterConstantDims(inArr):
    inArrCopy = np.copy(inArr)
    notTheSame = np.where(np.not_equal(inArrCopy[0], inArrCopy[1]))[0]
    for element in inArrCopy:
        inds = np.where(np.not_equal(inArrCopy[0], inArrCopy[1]))[0]
        notTheSame = np.intersect1d(notTheSame, inds)

    inArrCopy_filtered = inArrCopy[:,notTheSame]
    inArrCopy = None
    notTheSame = None # freeing up some memory
    return inArrCopy_filtered

In [7]:
split_num = 4
n_bins = 48


x_chunk_size = all_data.shape[1]/split_num
y_chunk_size = all_data.shape[2]/split_num
z_chunk_size = all_data.shape[3]/split_num

hist_data = []
minValue = np.amin(all_data)
maxValue = np.amax(all_data)

for scan in all_data:
    scan_hists = np.array([])
    i = 0
    for x_ind in range(0,split_num):
        for y_ind in range(0,split_num):
            for z_ind in range(0,split_num):
                chunk = scan[x_ind*x_chunk_size:(x_ind+1)*x_chunk_size, 
                             y_ind*y_chunk_size:(y_ind+1)*y_chunk_size,
                             z_ind*z_chunk_size:(z_ind+1)*z_chunk_size]
                #print chunk.shape
                scan_hists = np.append(scan_hists, 
                               np.histogram(np.array(chunk).flatten(), bins=n_bins, range = (minValue, maxValue))[0])
    # print scan_hists.shape, scan_hists.dtype
    hist_data.append(scan_hists)

hist_data = np.array(hist_data)
hist_data = filterConstantDims(hist_data)
# freeing up mem
scan_hists = None
chunk = None

In [8]:
np.save('hist_data_fine_big', hist_data)
hist_data.shape

(416, 972)

In [ ]:

import numpy as np

ages = np.load('ages.npy')
sexes = np.load('sexes.npy')
# flat_data_np = np.load('flat_data.npy')
#flat_data_np = np.load('hist_data.npy')

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

X = SelectKBest(chi2, k=200).fit_transform(flat_data_filtered, ages)
np.save('X', X)

In [ ]:
from sklearn.decomposition import PCA,IncrementalPCA,FactorAnalysis
import matplotlib.pyplot as plt
%matplotlib inline

try: 
    X = flat_data_filtered
except NameError:
    X = flat_data_np
    
#transformer = IncrementalPCA(batch_size=105, n_components=100)
#transformer = FactorAnalysis(n_components=100)
transformer = PCA(n_components=100)
model = transformer.fit(X, ages)
plt.plot(model.explained_variance_)

In [ ]:
transformedData = model.transform(X)
np.save('X',transformedData)

In [ ]:
#transformedData = np.load('transformedData2.npy')
import matplotlib.pyplot as plt
%matplotlib inline
dementiaClass = np.array(dementiaClass)
#plt.hist(dementiaClass.astype('int16'), bins=5)#[np.where(dementiaClass == 0)])
sum(dementiaClass < 2)
#dementiaClass.dtype
#min(transformedData[:,4]), max(transformedData[:,4])

#import pandas
#from collections import Counter
#counts = Counter(sexes)
#df = pandas.DataFrame.from_dict(counts, orient='index')
#df.plot(kind='bar', legend=False)
